In [ ]:
!pip install vosk
!sudo apt-get install ffmpeg


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 10.5 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22429 sha256=5ffd914b494637a36a18daace21de4141061a72ab4d1ce790796be215d5c92fb
  Stored in directory: /Users/asray/Library/Caches/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
Password:

In [ ]:
import queue
import sounddevice as sd
from vosk import Model, KaldiRecognizer
import json
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('scam_classifier')
tokenizer = BertTokenizer.from_pretrained('scam_classifier')

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits).item()
    return "spam" if predicted_class_id == 1 else "not_spam"

q = queue.Queue()

def callback(indata, frames, time, status):
    if status:
        print(status)
    q.put(bytes(indata))

model = Model("model")
recognizer = KaldiRecognizer(model, 16000)

with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                       channels=1, callback=callback):
    print('Listening... Press Ctrl+C to stop.')
    while True:
        data = q.get()
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            text = json.loads(result).get('text', '')
            if text:
                print(f"Transcribed Text: {text}")
                if predict(text) == "spam":
                    print("Scam Detected")


Listening... Press Ctrl+C to stop.
Transcribed Text: give me a million dollars
Scam Detected
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Google Speech Recognition could not understand audio
Transcribed Text: do I need a key ooh I like that one
Google Speech Recognition could not understand audio
Transcribed Text: good
